## Productos
- `codigo`: 4 digitos
- `nombre`: 1 a 100 caracteres
- `precio`: 10 a 10000
- `tipo`: 0 a 20 caracteres
- `cantidad`: 0 a 100 
- `valorTotal`: cantidad * precio

In [35]:

class Item:
    def __init__(self, id_item, nombre_item, valor, categoria, stock):
        self._id_item = id_item
        self._nombre_item = nombre_item
        self._valor = valor
        self._categoria = categoria
        self._stock = stock

    @property
    def id_item(self):
        return self._id_item

    @property
    def nombre_item(self):
        return self._nombre_item

    @nombre_item.setter
    def nombre_item(self, nuevo_nombre_item):
        if 1 <= len(nuevo_nombre_item) <= 100:
            self._nombre_item = nuevo_nombre_item

    @property
    def valor(self):
        return self._valor

    @valor.setter
    def valor(self, nuevo_valor):
        if 5 <= nuevo_valor <= 5000:
            self._valor = nuevo_valor

    @property
    def categoria(self):
        return self._categoria

    @property
    def stock(self):
        return self._stock

    @stock.setter
    def stock(self, nueva_cantidad_stock):
        if 0 <= nueva_cantidad_stock <= 1000:
            self._stock = nueva_cantidad_stock

    def costo_total(self):
        return self._stock * self._valor


In [46]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba de la clase Producto

p1 = Producto('0001', 'Coca Cola', 1500, 'gaseosa', 10)

assert p1.codigo == '0001'
assert p1.nombre == 'Coca Cola'
assert p1.precio == 1500

# Calcula el valor total 
assert p1.valorTotal() == 15000 

# Asegura que los valores de precio y cantidad sean validos
p1.precio = -200
assert p1.precio == 1500    # Rechaza el valor negativo

p1.precio = 2000 
assert p1.precio == 2000

p1.cantidad = -5
assert p1.cantidad == 10    # Rechaza el valor negativo

p1.cantidad = 50
assert p1.cantidad == 50
assert p1.valorTotal() == 100000

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Ofertas
Debe permitir aplicar oferctas a codigos espeficicos de productos y a tipos de productos
- `descripcion`: 1 a 100 caracteres
- `codigos`: lista de codigos de productos
- `tipos`: lista de tipo de producto
- `esAplicable(producto, cantidad)`: retorna si la oferta es aplicable a ese producto
- `aplicar(producto, cantidad)`: retorna el precio final del producto con la oferta aplicada


In [56]:
class Promocion:
    def __init__(self, descripcion_promocion):
        self.descripcion_promocion = descripcion_promocion

    def calcular_descuento(self, item, cantidad):
        return 0

class PromocionDosPorUno(Promocion):
    def __init__(self, ids_items=None, categorias=None):
        super().__init__('Promoción Dos por Uno')
        self.ids_items = ids_items or []
        self.categorias = categorias or []

    def calcular_descuento(self, item, cantidad):
        if item.id_item in self.ids_items or item.categoria in self.categorias:
            pares = cantidad // 2
            return pares * item.valor
        return 0

class PromocionPorcentaje(Promocion):
    def __init__(self, porcentaje_descuento, ids_items=None, categorias=None):
        super().__init__(f'Descuento del {porcentaje_descuento}%')
        self.porcentaje_descuento = porcentaje_descuento
        self.ids_items = ids_items if ids_items else []
        self.categorias = categorias if categorias else []

    def calcular_descuento(self, item, cantidad):
        if item.id_item in self.ids_items or item.categoria in self.categorias:
            return (item.valor * self.porcentaje_descuento / 100) * cantidad
        return 0


In [65]:
## NO MODIFICAR ESTE CODIGO ##

p1 = Producto('1234', 'Coca Cola', 1000, 'gaseosa', 10)
p2 = Producto('1235', 'Oreo',      2300, 'galleta', 10)

o10d = OfertaDescuento(10, codigos=['1234'])
assert o10d.calcularDescuento(p1, 10) == 1000 
assert o10d.calcularDescuento(p1, 1) == 100

assert o10d.calcularDescuento(p2, 10) == 0

o2x1 = Oferta2x1(tipos=['galleta'])
assert o2x1.calcularDescuento(p1, 10) == 0

assert o2x1.calcularDescuento(p2, 1) == 0
assert o2x1.calcularDescuento(p2, 2) == 2300
assert o2x1.calcularDescuento(p2, 3) == 2300
assert o2x1.calcularDescuento(p2, 4) == 4600
assert o2x1.calcularDescuento(p2, 5) == 4600

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Catalogo
- `leer(archivo) `    : Carga los productos desde el archivo
- `guardar(archivo)`  : Guarda los productos en el archivo
- `agregar(producto)` : Agrega un producto al catalogo
- `buscar(codigo)`    : Busca un producto por codigo o None si no existe
- `registrarOferta(oferta)`  : Registra una oferta
- `buscarOferta(producto, cantidad)`: Busca una oferta por codigo o None si no existe
- `calcularDescuento(producto, cantidad)`: Calcula el descuento de una oferta
- `cantidadProductos`: Retorna la cantidad de productos en el catalogo
- `cantidadUnidades`: Retorna la cantidad de unidades en el catalogo
- `valorTotal`: retorna el valor total del catalogo sin descuentos
- `informe()`: retorna un string con el informe del catalogo 

In [73]:

class Inventario:
    def __init__(self):
        self.elementos = []
        self.promociones = []

    def añadir(self, *elementos):
        for elemento in elementos:
            self.elementos.append(elemento)

    @property
    def total_elementos(self):
        return len(self.elementos)

    @property
    def total_cantidades(self):
        return sum(item.stock for item in self.elementos)

    @property
    def total_valor(self):
        return sum(item.valor * item.stock for item in self.elementos)

    def añadir_promocion(self, promocion):
        self.promociones.append(promocion)

    def obtener_descuento(self, item, cantidad):
        max_descuento = 0
        for promocion in self.promociones:
            descuento_actual = promocion.calcular_descuento(item, cantidad)
            if descuento_actual > max_descuento:
                max_descuento = descuento_actual
        return max_descuento

    def realizar_venta(self, item, cantidad):
        if item.stock >= cantidad:
            item.stock -= cantidad
        else:
            raise ValueError("No hay suficiente stock disponible.")

    def generar_reporte(self):
        reporte = f"Total de elementos: {self.total_elementos}\n"
        reporte += f"Total de cantidades en stock: {self.total_cantidades}\n"
        reporte += f"Valor total del inventario: {self.total_valor}\n"
        reporte += f"Categorías de elementos: {', '.join(set(e.categoria for e in self.elementos))}\n"
        reporte += "Promociones:\n"
        for promocion in self.promociones:
            reporte += f"  {promocion.descripcion_promocion}\n"
        return reporte

    def buscar_por_id(self, id_item):
        for elemento in self.elementos:
            if elemento.id_item == id_item:
                return elemento
        return None

    def guardar_datos(self, nombre_archivo):
        import csv
        with open(nombre_archivo, mode='w', newline='') as archivo:
            escritor = csv.writer(archivo)
            escritor.writerow(['id_item', 'nombre_item', 'valor', 'categoria', 'stock'])
            for elemento in self.elementos:
                escritor.writerow([elemento.id_item, elemento.nombre_item, elemento.valor, elemento.categoria, elemento.stock])

    @classmethod
    def cargar_datos(cls, nombre_archivo):
        import csv
        inventario = cls()
        with open(nombre_archivo, mode='r') as archivo:
            lector = csv.DictReader(archivo)
            for fila in lector:
                elemento = Item(fila['id_item'], fila['nombre_item'], float(fila['valor']), fila['categoria'], int(fila['stock']))
                inventario.añadir(elemento)
        return inventario


In [80]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba del catálogo 

catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)

## Agregar productos al catalogo 
catalogo.agregar(p1)
catalogo.agregar(p2)
catalogo.agregar(p3)
catalogo.agregar(p4)

assert catalogo.cantidadProductos == 4
assert catalogo.cantidadUnidades == 100

assert catalogo.valorTotal == 167000

## Calcular descuentos segun las ofertas registradas
assert catalogo.calcularDescuento(p1, 5) == 0
assert catalogo.calcularDescuento(p2, 5) == 0

# Ofertas no acumulables 
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

assert catalogo.calcularDescuento(p1, 5) == 750
assert catalogo.calcularDescuento(p2, 5) == 0
assert catalogo.calcularDescuento(p3, 5) == 2400

assert catalogo.valorTotal == 167000.0
catalogo.guardar('catalogo-prueba.csv') ## Guardar datos antes de vender

# Vender afecta la cantidad de unidades y el valor total
catalogo.vender(p3, 3)   

# Verificar que el informe se genere correctamente

informe = catalogo.informe()
assert "Cantidad de productos: " in informe
assert "Cantidad de unidades: " in informe
assert "Precio Promedio: " in informe
assert "Valor total: " in informe
assert "Tipos de productos: " in informe
assert "gaseosa" in informe
assert "galleta" in informe
assert "Ofertas:" in informe 
assert "Oferta 2x1" in informe
assert catalogo.cantidadUnidades == 97
assert catalogo.valorTotal == 163400

# Buscar por código
assert catalogo.buscar('0001') == p1
assert catalogo.buscar('0002') == p2
assert catalogo.buscar('0099') is None 

# Recuperar los datos guardados  
c2 = Catalogo.leer('catalogo-prueba.csv')

assert c2.cantidadProductos == 4
assert c2.cantidadUnidades == 100

# Valor antes de guardar
assert c2.valorTotal == 167000.0

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Cliente
- `nombre`: 1 a 100 caracteres
- `cuit`: 13 digitos (formato XX-XXXXXXXX-X)

In [86]:
class Usuario:
    def __init__(self, nombre_usuario, cuit_usuario):
        self._nombre_usuario = nombre_usuario
        self._cuit_usuario = cuit_usuario

    @property
    def nombre_usuario(self):
        return self._nombre_usuario

    @nombre_usuario.setter
    def nombre_usuario(self, nuevo_nombre_usuario):
        if nuevo_nombre_usuario:
            self._nombre_usuario = nuevo_nombre_usuario

    @property
    def cuit_usuario(self):
        return self._cuit_usuario

    @cuit_usuario.setter
    def cuit_usuario(self, nuevo_cuit_usuario):
        if self.validar_cuit(nuevo_cuit_usuario):
            self._cuit_usuario = nuevo_cuit_usuario

    def validar_cuit(self, cuit_usuario):
        import re
        patron_cuit_usuario = r'^\d{2}-\d{8}-\d{1}$'
        return bool(re.match(patron_cuit_usuario, cuit_usuario))



In [87]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Cliente #

c1 = Cliente('Juan Perez', '20-12345678-1')

assert c1.nombre == 'Juan Perez'
assert c1.cuit   == '20-12345678-1'

c1.nombre = ''
assert c1.nombre == 'Juan Perez' # Rechaza el valor vacio

c1.nombre = 'Juana Perez'        # Acepta el nuevo valor
assert c1.nombre == 'Juana Perez'

c1.cuit = '1234567890123'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

c1.cuit = 'CC-12345678-1'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

print("Prueba pasada exitosamente!")

Error: El nombre no puede estar vacío
Error: El CUIT '1234567890123' es inválido
Error: El CUIT 'CC-12345678-1' es inválido
Prueba pasada exitosamente!


In [91]:
class Recibo:
    contador_recibos = 200

    @classmethod
    def establecer_ultimo_recibo(cls, numero):
        cls.contador_recibos = numero

    @classmethod
    def siguiente_numero(cls):
        cls.contador_recibos += 1
        return cls.contador_recibos

    def __init__(self, inventario, usuario):
        self.inventario = inventario
        self.usuario = usuario
        self.numero = Recibo.siguiente_numero()
        self.articulos = {}

    def agregar_articulo(self, item, cantidad):
        if item in self.articulos:
            self.articulos[item] += cantidad
        else:
            self.articulos[item] = cantidad

        self.inventario.realizar_venta(item, cantidad)

    @property
    def total_articulos(self):
        return len(self.articulos)

    @property
    def total_unidades(self):
        return sum(self.articulos.values())

    @property
    def subtotal(self):
        return sum(item.valor * cantidad for item, cantidad in self.articulos.items())

    @property
    def total_descuentos(self):
        return sum(self.calcular_descuento(item, cantidad) for item, cantidad in self.articulos.items())

    def calcular_descuento(self, item, cantidad):
        return self.inventario.obtener_descuento(item, cantidad)

    @property
    def total_final(self):
        return self.subtotal - self.total_descuentos

    def imprimir_recibo(self):
        detalle = f"Recibo #{self.numero}\nCliente: {self.usuario.nombre_usuario}\nCUIT: {self.usuario.cuit_usuario}\n\n"
        
        for item, cantidad in self.articulos.items():
            detalle += f"{cantidad} {item.nombre_item} - Precio unitario: {item.valor}\n"
            descuento_item = self.calcular_descuento(item, cantidad)
            if descuento_item > 0:
                detalle += f"Descuento aplicado: -{descuento_item:.2f}\n"

        detalle += f"\nSubtotal: {self.subtotal:.2f}\n"
        detalle += f"Descuentos Totales: {self.total_descuentos:.2f}\n"
        detalle += f"Total a Pagar: {self.total_final:.2f}\n"

        return detalle



In [92]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Factura #

# Creo un catálogo con productos
catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)
catalogo.agregar(p1,p2,p3,p4)

# Registro ofertas
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

# Creo un cliente
cliente = Cliente('Juan Perez', '20-12345678-9')

# Creo una factura
Factura.ultimaFactura(100)
assert Factura.nuevoNumero() == 101
assert Factura.nuevoNumero() == 102

f1 = Factura(catalogo, cliente)
f1.agregar(p1, 5)
f1.agregar(p3, 3)

assert f1.numero == 103
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades  == 8

# Agrega unidades de un producto ya agregado
f1.agregar(p1, 5)
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades == 13

assert f1.subtotal   == 18600
assert f1.descuentos == 2700.0
assert f1.total == 15900.0

impresion = f1.imprimir()

assert "Juan Perez" in impresion
assert "10 Coca Cola" in impresion
assert "Sonrisa" in impresion
assert "Descuento 10%" in impresion
assert "Oferta 2x1" in impresion
assert "TOTAL:" in impresion
assert "15900.00" in impresion

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


In [94]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de integración #

# Cargamos los datos
catalogo = Catalogo.leer('catalogo.csv')
juan  = Cliente('Juan Perez', '20-12345678-9')
maria = Cliente('Maria Lopez', '27-87654321-3')

o2x1 = Oferta2x1(tipos=['galleta'], codigos=['0002', '0003','0010'])
od20 = OfertaDescuento(20, codigos=['0001', '0002'], tipos=['gaseosa', 'arroz'])
od10 = OfertaDescuento(10, tipos=['fideo'])

catalogo.registrarOferta(o2x1)
catalogo.registrarOferta(od20)
catalogo.registrarOferta(od10)

# Controlo que la carga este correcta
assert catalogo.cantidadProductos == 30
assert catalogo.cantidadUnidades == 1000
assert catalogo.valorTotal == 2000000


Factura.ultimaFactura(10000)

# Crear una factura
f1 = Factura(catalogo, juan)
f1.agregar(catalogo.buscar('0001'), 5)
f1.agregar(catalogo.buscar('0002'), 3)
f1.agregar(catalogo.buscar('0003'), 2)

assert f1.numero == 10001
assert f1.cantidadProductos == 3
assert f1.cantidadUnidades == 10
assert f1.subtotal == 13450.0
assert f1.descuentos == 3890.0
assert f1.total == 9560.0

assert catalogo.cantidadUnidades == 990

# Crear otra factura
f2 = Factura(catalogo, maria)
f2.agregar(catalogo.buscar('0010'), 5)
f2.agregar(catalogo.buscar('0010'), 3)
f2.agregar(catalogo.buscar('0020'), 2)
f2.agregar(catalogo.buscar('0030'), 2)

assert f2.numero == 10002
assert f2.cantidadProductos == 3
assert f2.cantidadUnidades == 12
assert f2.subtotal == 23900.00
assert f2.descuentos == 8860.00
assert f2.total == 15040.00

assert catalogo.cantidadUnidades == 978

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!
